# GeoJupyter demo

## Today: [🔗JupyterGIS](https://jupytergis.readthedocs.io/)

JupyterGIS is a **real-time collaborative** Geographical Information System (GIS) environment in JupyterLab.

You can [🔗try it right now in JupyterLite](https://jupytergis.readthedocs.io/en/latest/lite/lab/)!

Let's explore some functionality together (based on [🔗Carl Boettiger](https://ourenvironment.berkeley.edu/people/carl-boettiger)'s [🔗ESPM-288 course](https://espm-288.carlboettiger.info/)). We'll explore whether neighborhoods that were highly-rated (A) under the disciminatory 1930s practice of [🔗redlining](https://en.wikipedia.org/wiki/Redlining) are greener today than neighborhoods graded D?

### Constants

Some variables that will be used throughout the Notebook.

In [ ]:
from pathlib import Path

DATA_DIR = Path().cwd() / "data"
INEQUALITY_GEOJSON_FILE = DATA_DIR / "redlining_newhaven_ct.geojson"
NDVI_GEOTIFF_FILE = DATA_DIR / "ndvi.tif"
INEQUALITY_NDVI_GEOJSON_FILE = DATA_DIR / "redlining_ndvi_newhaven_ct.geojson"

### Set up a JupyterGIS project

We can build a JupyterGIS project from scratch in Python. Let's start by adding an OpenStreetMap basemap and displaying the widget in a side panel.

In [ ]:
from jupytergis import GISDocument

jgis_project = GISDocument()
jgis_project.add_raster_layer(
    url="https://tile.openstreetmap.org/{z}/{x}/{y}.png",
    name="Basemap",
)

jgis_project.sidecar()

### Get historical redlining data

We're using [🔗DuckDB](https://duckdb.org/) to connect to a [🔗geopackage](https://www.geopackage.org/) dataset containing data about redlining, and filter that data to select residential neighborhoods in New Haven, Connecticut, USA.

In [ ]:
import ibis
from ibis import _


con = ibis.duckdb.connect(extensions=["spatial"])

redlines = (
    con
    .read_geo("/vsicurl/http://dsl.richmond.edu/panorama/redlining/static/mappinginequality.gpkg")
    .filter(_.city == "New Haven", _.residential)
)

new_haven_redlining =  redlines.execute().set_crs("EPSG:4326")
# TODO: Set a numeric grade for graduated symbology

new_haven_redlining.to_file(INEQUALITY_GEOJSON_FILE, engine="fiona")

new_haven_bbox = new_haven_redlining.total_bounds

#### Explore the data

Let's explore the data a little bit. After running the cell below, **right-click the "New Haven neighborhood redlining" layer** in the JupyterGIS interface, and **select "Zoom to layer"**. 

In [ ]:
jgis_project.add_geojson_layer(
    path=INEQUALITY_GEOJSON_FILE,
    name="New Haven neighborhood redlining",
);

Now, with the "New Haven neighborhood redlining" layer selected, **click the `i` (identify) icon in the toolbar** at the top of the JupyterGIS interface.

Select some neighborhoods and view their "Grade" and "Category" attributes.

TODO: Symbologize on numeric grade

### Calculate NDVI

We're going to calculate NDVI from Sentinel-2 data.

#### Open Sentinel-2 data

We are using a [🔗STAC catalog](https://stacspec.org/en) to locate the data files we're interested in (covering New Haven during Summer 2024, with <20% cloud cover) and opening them as an Xarray DataSet.

In [ ]:
import odc.stac
from pystac_client import Client

items = Client.open(
    "https://earth-search.aws.element84.com/v1"
).search(
    collections = ['sentinel-2-l2a'],
    bbox=new_haven_bbox,
    datetime = "2024-06-01/2024-09-01",
    query={"eo:cloud_cover": {"lt": 20}}
).item_collection()

data = odc.stac.load(
    items,
    bands=["nir08", "red"],
    bbox=new_haven_bbox,
    resolution=10,
    groupby="solar_day",
    chunks = {},  # this tells odc to use dask
)
data

#### Do the NDVI calculation

In [ ]:
ndvi = (
    (data.nir08 - data.red) / (data.red + data.nir08)
).median(
    "time",
    keep_attrs=True,
).where(
    ndvi < 1
).compute()

ndvi.plot.imshow()

#### Save the NDVI raster to file

In [ ]:
import rioxarray

ndvi.rio.reproject(
    "EPSG:4326",
).rio.to_raster(
    raster_path=NDVI_GEOTIFF_FILE, 
    driver="COG",
)

#### Explore the data

Let's explore the data in JupyterGIS again.
This time, we'll add the layer with the GUI.

If the "identify" tool is still active, click the `i` icon in the toolbar again to disable it.

Now, **click the `+` icon in the toolbar** to open the new layer interface.
**Select "Add Raster Layer", then "New GeoTiff Layer"**.

**Select "Browse Server Files"** and then **navigate to `ndvi.tif` in the module 6 data directory** (`workshop-open-source-geospatial/modules/06-geojupyter/data/ndvi.tif`).

Click **Select**.

**Set the "Min" field to `0` and "Max" to `1`**.

**Uncheck "Normalize"**.

Scroll down to **input the layer name as "NDVI"**.

Finally, **right-click the "NDVI" layer** and **select "Edit Symbology"**. The symbology menu may take a moment to load. Be patient! **Select "Classify" then click "OK".**

The brighter areas have a higher NDVI value, and the darker areas have a lower one.

We can use the identify tool (`i` icon in the toolbar) to explore the raw values.

### Calculating mean NDVI for each New Haven neighborhood

To find out whether neighborhoods graded "A" are greener than neighborhoods graded "D", we'll calculate the mean NDVI for each neighborhood using [🔗exactextract](https://isciences.github.io/exactextract/background.html), which is known for its capability to include fractional grid cells in its calculation (as opposed to other tools, where a cell is binary, either in or out).

In [ ]:
from exactextract import exact_extract

new_haven_redlining_and_ndvi = exact_extract(
    NDVI_GEOTIFF_FILE,
    new_haven_redlining,
    "mean_ndvi=mean",
    include_geom = True,
    include_cols=["label", "grade", "city", "fill"],
    output="pandas",
)

new_haven_redlining_and_ndvi.set_crs(
    "EPSG:4326"
).to_file(INEQUALITY_NDVI_GEOJSON_FILE, engine="fiona")

In [ ]:
# jgis_project.add_geojson_layer(
#     path=INEQUALITY_NDVI_GEOJSON_FILE,
#     name="New Haven neighborhood redlining w/ NDVI",
# )  # TODO

## Future

### Story maps / "scrolly telling"

Story map support for JupyterGIS is in progress.

We anticipate working with the [🔗MyST](https://mystmd.org/) and [🔗Closeread](https://closeread.dev/) developers to develop interactive scrollytelling experiences in MyST Markdown documents.

### "microgis" (placeholder name)

We're working on a [🔗project](https://github.com/geojupyter/jupyter-microgis) to provide an instant layered visual environment for any number of Python datasets (starting with rioxarray DataArrays and GeoPandas GeoDataFrames) in a widget.
The goal is to minimize time-to-visualization.

It would provide sensible default symbology choices, and customization would be available with as-needed complexity.
In other words, you shouldn't need to learn a complex symbology expression language when your needs are simple, but complex expression is available if you need it.

```python
from microgis import explore


explore(
    da1, da2, gdf1,
    {
        "data": gdf2,
        "symbology": {
            "choropleth": {
                "steps": 11,
                "classification": "natural",
            },
        },
    },
)
```

### More!

:::{image} https://geojupyter.org/assets/images/community-diagram.svg
:width: 400px
:align: center
:::

GeoJupyter's priorities are broad, and are based on our community's needs. We can only know what those needs are if you join us!

Please join the [🔗Jupyter Zulip](https://jupyter.zulipchat.com) today and find us in the `#geojupyter` channel!

